First We read the databases that we stored (sample1:tweets_h , sample2: tweets_geo)

In [1]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client['twitter_geo_sample']
tweets_geo = db.tweets_geo


db = client['twitter_h_sample']
tweets_h = db.tweets_h

Sample1: calculating the number of tweets that do not contain any URL

In [2]:
tweets_h.count_documents({'entities.urls': []})

2444

Sample2: calculating the number of tweets that do not contain any URL

In [3]:
tweets_geo.count_documents({'entities.urls': []})

1921

Sample1: calculate number of tweets that are (or contain) retweets

In [4]:
x = tweets_h.count_documents({'retweeted_status': {"$exists": False}})
y = tweets_h.count_documents({'retweeted':  False})

print(x)
print(y)

613
2943


Sample2: calculate number of tweets that are (or contain) retweets

In [5]:
x = tweets_geo.count_documents({'retweeted_status': {"$exists": False}})
y = tweets_geo.count_documents({'retweeted':  False})

print(x)
print(y)

3000
3000


Sample1: calculate number of tweets with each of the specified languages

In [6]:
languages = ['en','fr','de','it','es','zh','ko','pt','tr','fa','sv','nl','ja','hi','th','pl','el','ar','ca','ru','ro','cs']

for l in languages:
    x = tweets_h.count_documents({'lang': l})
    print(x)




1629
233
41
56
492
5
2
32
44
4
1
12
13
69
55
1
2
18
24
4
0
0


Sample2: calculate number of tweets with each of the specified languages

In [7]:
languages = ['en','fr','de','it','es','zh','ko','pt','tr','fa','sv','nl','ja','hi','th','pl','el','ar','ca','ru','ro','cs']

for l in languages:
    x = tweets_geo.count_documents({'lang': l})
    print(x)


511
734
490
334
158
0
0
104
81
5
1
9
11
1
1
5
1
31
3
4
2
4


Sample1: finding 20 most frequent hashtags

To do that:
- we first save all the hashtags in an array ( I took all the hashtags from the tweets and also from the retweet field and quoted field). 
- Then we start from the first hashtag and count the number of the same hashtag through the whole array. We also remove all the hashtags that were the same as the first one. 
- Then we go to the next hashtag and continue it until we reach the end of the array
- At the end we sort the hashtags based on their frequency

In [8]:
import numpy

list_hashtags = []


docs1 = list(tweets_h.find({'entities.hashtags.text': {"$exists": True}}))

for doc in docs1:
    for i in range(0, len(doc['entities']['hashtags'])):
        #print("\n",doc['entities']['hashtags'][i]['text'])
        list_hashtags.append(doc['entities']['hashtags'][i]['text'])
        
docs2 = list(tweets_h.find({'extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs2:
    for i in range(0, len(doc['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['extended_tweet']['entities']['hashtags'][i]['text'])
        
docs3 = list(tweets_h.find({'quoted_status.entities.hashtags.text': {"$exists": True}}))

for doc in docs3:
    for i in range(0, len(doc['quoted_status']['entities']['hashtags'])):
        list_hashtags.append(doc['quoted_status']['entities']['hashtags'][i]['text'])
 
docs4 = list(tweets_h.find({'quoted_status.extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs4:
    for i in range(0, len(doc['quoted_status']['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['quoted_status']['extended_tweet']['entities']['hashtags'][i]['text'])

docs5 = list(tweets_h.find({'retweeted_status.entities.hashtags.text': {"$exists": True}}))

for doc in docs5:
    for i in range(0, len(doc['retweeted_status']['entities']['hashtags'])):
        list_hashtags.append(doc['retweeted_status']['entities']['hashtags'][i]['text'])
 
docs6 = list(tweets_h.find({'retweeted_status.extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs6:
    for i in range(0, len(doc['retweeted_status']['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['retweeted_status']['extended_tweet']['entities']['hashtags'][i]['text'])

l = len(list_hashtags)
c = 0
hashtags = [];
count = [];

while l > 0:
    hashtag = list_hashtags[0]
    
    hashtags.append(hashtag)
    
    while True:
        try:
            list_hashtags.remove(hashtag)
        except ValueError:
            break
    
    count.append(l - len(list_hashtags))

    l = len(list_hashtags)
        

sorted_arg = numpy.argsort(count)
sorted_arg = sorted_arg[::-1]
sorted_hashtags = []
sorted_count = []
for i in sorted_arg:
    sorted_hashtags.append(hashtags[i])
    sorted_count.append(count[i])
    
print(sorted_hashtags)
print(sorted_count)

['COVID19', 'coronavirus', 'Coronavirus', 'Covid19', 'Corona', 'CoronaVirus', 'covid19', 'CoronaUpdate', 'IndiaFightsCorona', 'Covid_19', 'CoronaLockdown', 'Taiwan', 'COVID', 'COVID2019', 'StayHomeIndia', 'China', 'Cuba', 'โควิด_19', 'BelieveItOrNot', 'StayAtHomeAndStaySafe', 'StayHome', 'โควิด', 'CoronavirusOutbreak', 'โควิด19เราต้องรอด', 'Trump', 'Yemen', 'CORONAVIRUS', 'France', 'RestezChezVous', 'chloroquine', 'Wuhan', 'SocialDistancing', 'koronavirus', 'USNSMercy', 'COVIDー19', 'USA', 'India', '28Mar', 'CARESAct', 'AkshayKumar', 'BREAKING', 'StayAtHome', 'US', 'confinement', 'VirusCorona', 'Italy', 'Macron', 'corona', 'Covid', 'Michigan', 'BuhariResign', 'lockdown', 'SARSCoV2', 'COVID19france', 'masks', 'StayHomeSaveLives', 'TataTrusts', 'EsteVirusLoParamosUnidos', 'RatanTata', 'โควิด19', 'Belgique', 'Pakistan', 'QuédateEnCasa', 'México', 'stayathome', 'Lockdown', 'Raoult', 'Iraq', 'CuartoMilenio', 'Israel', 'DSSServesMankind', 'CONFINEMENTJOUR11', 'virus', 'PMCARES', 'SaturdayMorn

Sample2: finding 20 most frequent hashtags

To do that:
- we first save all the hashtags in an array ( I took all the hashtags from the tweets and also from the retweet field and quoted field). 
- Then we start from the first hashtag and count the number of the same hashtag through the whole array. We also remove all the hashtags that were the same as the first one. 
- Then we go to the next hashtag and continue it until we reach the end of the array
- At the end we sort the hashtags based on their frequency

In [9]:
import numpy

list_hashtags = []


docs1 = list(tweets_geo.find({'entities.hashtags.text': {"$exists": True}}))

for doc in docs1:
    for i in range(0, len(doc['entities']['hashtags'])):
        #print("\n",doc['entities']['hashtags'][i]['text'])
        list_hashtags.append(doc['entities']['hashtags'][i]['text'])
        
docs2 = list(tweets_geo.find({'extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs2:
    for i in range(0, len(doc['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['extended_tweet']['entities']['hashtags'][i]['text'])
        
docs3 = list(tweets_geo.find({'quoted_status.entities.hashtags.text': {"$exists": True}}))

for doc in docs3:
    for i in range(0, len(doc['quoted_status']['entities']['hashtags'])):
        list_hashtags.append(doc['quoted_status']['entities']['hashtags'][i]['text'])
 
docs4 = list(tweets_geo.find({'quoted_status.extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs4:
    for i in range(0, len(doc['quoted_status']['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['quoted_status']['extended_tweet']['entities']['hashtags'][i]['text'])

docs5 = list(tweets_geo.find({'retweeted_status.entities.hashtags.text': {"$exists": True}}))

for doc in docs5:
    for i in range(0, len(doc['retweeted_status']['entities']['hashtags'])):
        list_hashtags.append(doc['retweeted_status']['entities']['hashtags'][i]['text'])
 
docs6 = list(tweets_geo.find({'retweeted_status.extended_tweet.entities.hashtags.text': {"$exists": True}}))

for doc in docs6:
    for i in range(0, len(doc['retweeted_status']['extended_tweet']['entities']['hashtags'])):
        list_hashtags.append(doc['retweeted_status']['extended_tweet']['entities']['hashtags'][i]['text'])

l = len(list_hashtags)
c = 0
hashtags = [];
count = [];

while l > 0:
    hashtag = list_hashtags[0]
    
    hashtags.append(hashtag)
    
    while True:
        try:
            list_hashtags.remove(hashtag)
        except ValueError:
            break
    
    count.append(l - len(list_hashtags))

    l = len(list_hashtags)
        

sorted_arg = numpy.argsort(count)
sorted_arg = sorted_arg[::-1]
sorted_hashtags = []
sorted_count = []
for i in sorted_arg:
    sorted_hashtags.append(hashtags[i])
    sorted_count.append(count[i])
    
print(sorted_hashtags)
print(sorted_count)

['CoronavirusDelCastigoMinore', 'COVID19', 'coronavirus', 'Covid_19', 'covid19', 'stayhome', 'stayathome', 'Coronavirus', 'CoronavirusOutbreak', 'RestezChezVous', 'Coronakrise', 'Corona', 'switzerland', 'confinement', 'Confinementjour12', 'beautiful', 'Germany', 'horizon', 'love', 'TwitchStreamAid', 'architecture', 'CoronaLockdown', 'sunset', 'SocialDistancing', 'schweiz', 'wandern', 'alpok', 'berg', 'grünten', 'Burgberg', 'sonthofen', 'corona', 'alps', 'rettenberg', 'alpen', 'hegyek', 'covid_19', 'weather', 'bergen', 'photography', 'allgäu', 'wanderlust', 'nature', 'sky', 'chloroquine', 'stayhealthy', 'weekend', 'quarantine', 'Switzerland', 'EarthHour', 'art', 'SilverTopper', 'Nuhr', 'picoftheday', 'Europe', 'Italy', 'photooftheday', 'zurich', 'Suisse', 'photo', 'CoronaInfoCH', 'restateacasa', 'perspective', 'solidarité', 'zuhausebleiben', 'TeamOM', 'wissenschaftstheorie', 'YoLoCiq', 'santé', 'COVIDー19', 'resterchezsoi', 'Cameroun', 'Covid19', 'Khaled_Drareni', 'running', 'Ditfurth', 